## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# ! wget -O lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

In [3]:
lenta = pd.read_csv('lenta-ru-news.csv', usecols=['title', 'text', 'topic'])
lenta = lenta[lenta['topic'].notna()]

C:\Users\User\AppData\Local\Temp\ipykernel_3160\1630857117.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  lenta = pd.read_csv('lenta-ru-news.csv', usecols=['title', 'text', 'topic'])


In [4]:
lenta.shape

(738973, 3)

In [5]:
label_dict = {}

for i, topic in enumerate(lenta['topic'].unique()):
    label_dict[topic] = i

label_dict

{'Библиотека': 0,
 'Россия': 1,
 'Мир': 2,
 'Экономика': 3,
 'Интернет и СМИ': 4,
 'Спорт': 5,
 'Культура': 6,
 'Из жизни': 7,
 'Силовые структуры': 8,
 'Наука и техника': 9,
 'Бывший СССР': 10,
 'Дом': 11,
 'Сочи': 12,
 'ЧМ-2014': 13,
 'Путешествия': 14,
 'Ценности': 15,
 'Легпром': 16,
 'Бизнес': 17,
 'МедНовости': 18,
 'Оружие': 19,
 '69-я параллель': 20,
 'Культпросвет ': 21,
 'Крым': 22}

In [6]:
# from nltk import tokenize

# tokenizer = tokenize.NLTKWordTokenizer()

In [7]:
# from nltk.corpus import stopwords
# from string import punctuation

# noise = stopwords.words('russian') + list(punctuation)

In [8]:
# import pymorphy3
# morph = pymorphy3.MorphAnalyzer()

# def morphling_lemmatizer(word):
#     parsed_word = morph.parse(word)[0]
#     lemma = parsed_word.normal_form

#     return lemma

# morphling_lemmatizer('деревьев')

In [9]:
# from nltk.stem import SnowballStemmer

# snowball_stemmer = SnowballStemmer("russian")

# def ru_stemmer(word):
#     return snowball_stemmer.stem(word)

In [10]:
# def preprocess(sentence):
#     token_sentence = tokenizer.tokenize(sentence)   
#     clean_tokens = [token for token in token_sentence if token not in noise]
#     #lemma_sentence = [morphling_lemmatizer(token) for token in clean_tokens]  
#     stemmed_sentence = [ru_stemmer(token) for token in clean_tokens]
#     return stemmed_sentence

# preprocess('как купить много деревьев')

In [11]:
# %%time

# from tqdm import tqdm

# title_preprocessed = [str(sentence).lower() for sentence in tqdm(lenta['title'], desc='Preprocessing titles ...')]
# text_preprocessed = [str(sentence).lower() for sentence in tqdm(lenta['text'], desc='Preprocessing text ...')]

# preprocessed_lenta = pd.DataFrame({
#     'title' : title_preprocessed,
#     'text' : text_preprocessed,
#     'topic' : lenta['topic']
# })

# preprocessed_lenta.to_csv('preprocessed_lenta.csv')

In [12]:
lenta['label'] = lenta['topic'].apply(lambda x: label_dict[x])

In [13]:
lenta['topic'].value_counts()

topic
Россия               160445
Мир                  136621
Экономика             79528
Спорт                 64413
Культура              53797
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44663
Из жизни              27605
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64

In [60]:
from sklearn.model_selection import train_test_split

def get_test_and_save_train(X_df, y_df, train_path):
    X, y = X_df.tolist(), y_df.tolist()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, shuffle=True)

    with open(train_path+'.txt', 'w', encoding='utf-8') as file:
        for X_entry, y_entry in zip(X_train, y_train):
            X_entry = str(X_entry).replace('\n', ' ').replace('\r', ' ')
            file.write('__label__' + str(y_entry) + ' ' + X_entry)
            file.write('\n')

    return X_test, y_test
            
X_test, y_test = get_test_and_save_train(lenta['title'], lenta['label'], train_path='lenta_train')

In [47]:
import fasttext

ft_model = fasttext.train_supervised(
    input='lenta_train.txt',
    label='__label__',
    lr=0.5,
    epoch=75,
    wordNgrams=2, 
    dim=200,
    thread=2,
    verbose=3000
)

In [55]:
label = ft_model.predict('армия')[0]
label

('__label__2',)

In [79]:
predicted_labels = ft_model.predict(X_test)[0]
predicted_labels = [int(label[0][9:]) for label in predicted_labels]

In [82]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

print("Accuracy score: ", accuracy_score(y_test, predicted_labels))
print("Precision score: ", precision_score(y_test, predicted_labels, average='weighted'))
print("Recall score: ", recall_score(y_test, predicted_labels, average='weighted'))
print("f1-score: ", f1_score(y_test, predicted_labels, average='weighted'))

report = classification_report(y_test, predicted_labels)
print("\nClassification Report:\n", report)

Accuracy score:  0.7436560862599056
Precision score:  0.748404059050117


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Recall score:  0.7436560862599056
f1-score:  0.743832903247207

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.58      0.73        19
           1       0.73      0.77      0.75     40517
           2       0.73      0.78      0.76     34008
           3       0.76      0.78      0.77     19749
           4       0.72      0.64      0.68     11207
           5       0.94      0.92      0.93     16047
           6       0.80      0.80      0.80     13469
           7       0.57      0.43      0.49      6938
           8       0.56      0.35      0.43      4950
           9       0.75      0.75      0.75     13223
          10       0.78      0.80      0.79     13230
          11       0.77      0.69      0.73      5368
          12       0.00      0.00      0.00         0
          14       0.61      0.47      0.53      1561
          15       0.79      0.60      0.68      2014
          16       0.60      0.11      0.19    

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\metrics\_classification.py:1517: